# 📝 Building a Notion Integration with Python 🚀

Hey there! Welcome to this fun guide where we'll learn how to connect your Python applications with Notion! We'll create a powerful integration that lets you programmatically interact with your Notion workspace. 🌟

## 🎯 What We'll Build

We're going to create a Notion integration that can:
1. 📊 Read and write to Notion databases
2. 📑 Create and update pages
3. 🔄 Sync data between Notion and your applications

## ✅ Prerequisites

Before we dive in, make sure you have:
- 📓 A Notion account
- 🎯 A database in Notion that you want to work with

## 🔑 Part 1: Setting Up Your Notion Integration

First, let's get you set up with the necessary credentials to talk to Notion! 

### 1. Create a Notion Integration

1. 🌐 Go to [Notion's Integration page](https://www.notion.so/my-integrations)
2. 👆 Click the "New integration" button
3. 📝 Give your integration a name (like "My Python Integration")
4. 🎨 Choose an icon and color if you want (make it pretty! ✨)
5. 💫 Select the workspace where you want to use the integration
6. 🎁 Click "Submit" to create your integration

You'll receive a secret token that looks something like this:
`secret_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`

⚠️ Important Security Note! ⚠️
Keep this token safe and never share it publicly! It's like the key to your Notion kingdom! 🏰

### 2. Get Your Database ID

Now we need to find the ID of the database you want to work with:

1. 📊 Open your Notion database in the browser
2. 🔗 Look at the URL, it will look something like:
   `https://www.notion.so/workspace/[database-id]?v=...`
3. 📋 Copy the database ID part (it's a string of characters between the last '/' and '?')

For example, from this URL:
`https://www.notion.so/workspace/7749c8c4a3f34c8c8c8c8c8c8c8c8c8c?v=...`
The database ID would be: `7749c8c4a3f34c8c8c8c8c8c8c8c8c8c`

### 3. Connect Your Database

One last important step!

1. 📱 Go to your database in Notion
2. ⚙️ Click the '...' menu in the top right
3. 👥 Look for "Add connections"
4. 🔍 Find your integration and click to connect
5. ✅ Click "Confirm" to give your integration access

Now your integration has permission to access this database! 🎉

## 🛠️ Part 2: Setting Up Your Python Environment

Let's get your Python environment ready with all the tools we need! We'll use the official Notion SDK for Python to make our lives easier. 🐍

First, let's install the required package:

In [29]:
!pip install notion-client openai


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [30]:
import os
from openai import OpenAI
from notion_client import Client
from dotenv import load_dotenv
from pprint import pprint # Pretty printing json

load_dotenv()

True

## 🔐 Part 3: Setting Up Environment Variables

Let's keep our secrets safe! We'll use a `.env` file to store our sensitive information. 

Create a file named `.env` in your project directory and add these lines:

In [31]:
NOTION_API_KEY = "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"
ROUTINES_DB_ID = "19180247952680c4865ec31554077721"
ACTIVITY_DB_ID = "15f8024795268117b86be7403cfd4b7d"
TEST_DB = "19f802479526804dad59c602983df30a"

## 📊 Part 4: Basic Notion Operations

Let's look at some common operations you can do with your Notion integration! Each operation is like a different superpower for your application! 🦸‍♂️

### Reading from a Database 📖

The Notion API lets you query your database like a pro! You can:
- 🔍 Filter entries based on properties
- 📋 Sort entries in any order
- 📝 Get specific properties only

In [32]:
client = Client(auth=NOTION_API_KEY)

response = client.databases.query(
    database_id=TEST_DB,
)

for page in response["results"]:
    page_id = response["results"][0]["id"]
    print(page_id)
    # Retrieve the blocks of the page
    page_content = client.blocks.children.list(block_id=page_id)

    # Print the content
    for block in page_content["results"]:
        pprint(block["paragraph"])

19f80247-9526-80b8-9d18-de0158ff23da


In [33]:
!pip install markdown-it-py


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [34]:
import markdown_it
from markdown_it.token import Token
from notion_client import Client

# Initialize Notion client
notion = Client(auth=NOTION_API_KEY)

# Function to convert Markdown to Notion blocks
def markdown_to_notion_blocks(md_text):
    md = markdown_it.MarkdownIt()
    tokens = md.parse(md_text)

    blocks = []
    for token in tokens:
        if token.type == "paragraph_open":
            blocks.append({"type": "paragraph", "paragraph": {"rich_text": []}})
        elif token.type == "inline":
            text_content = token.content
            if blocks and blocks[-1]["type"] == "paragraph":
                blocks[-1]["paragraph"]["rich_text"].append({"type": "text", "text": {"content": text_content}})
        elif token.type.startswith("heading"):
            level = int(token.tag[1])  # Extract heading level (e.g., h1 -> 1)
            blocks.append({
                "type": f"heading_{level}",
                f"heading_{level}": {"rich_text": [{"type": "text", "text": {"content": token.content}}]}
            })
        elif token.type == "bullet_list_open":
            blocks.append({"type": "bulleted_list_item", "bulleted_list_item": {"rich_text": []}})
        elif token.type == "text":
            if blocks and "rich_text" in blocks[-1].get(blocks[-1]["type"], {}):
                blocks[-1][blocks[-1]["type"]]["rich_text"].append({"type": "text", "text": {"content": token.content}})

    return blocks

In [35]:
def add_markdown_to_notion(page_id, md_text):
    blocks = markdown_to_notion_blocks(md_text)
    
    # Send blocks to Notion
    notion.blocks.children.append(
        block_id=page_id,
        children=blocks
    )

# Example Usage
page_id = "19f80247-9526-80b8-9d18-de0158ff23da"
markdown_content = """
# My Notion Page
This is a **bold** text and _italic_ text.

- Bullet list item 1
- Bullet list item 2
"""

add_markdown_to_notion(page_id, markdown_content)

In [36]:

client = Client(auth=NOTION_API_KEY)

response = client.databases.query(
    database_id=ACTIVITY_DB_ID,
    filter={
        "property": "Status",
        "status": {
            "does_not_equal": "Done"
        }
    }
)

pages = []

for page in response["results"]:
    properties = page["properties"]
    page_dict = {
        "name": properties.get("Name", {}).get('title', [{}])[0].get('plain_text', ""),
        "priority": (properties.get("Priority", {}).get('select') or {}).get('name', ""),
        "deadline": (properties.get("Deadline", {}).get('date') or {}).get('start', ""),
        "status": (properties.get("Status", {}).get('status') or {}).get('name', ""),
        "size": (properties.get("Size", {}).get('select') or {}).get('name', ""),
        "area": (properties.get("Area", {}).get('select') or {}).get('name', ""),
    }
    pages.append(page_dict)
pprint(pages)

APIResponseError: Could not find database with ID: 15f80247-9526-8117-b86b-e7403cfd4b7d. Make sure the relevant pages and databases are shared with your integration.

In [6]:
async def get_activities(status_filter="not_done"):
    """
    Fetch activities from Notion database with optional status filtering
    
    Parameters:
        status_filter (str): Filter type for status. Options:
            - "not_done": Returns activities not marked as Done (default)
            - "done": Returns only Done activities
            - "all": Returns all activities regardless of status
            
    Returns:
        list: List of dictionaries containing activity details
    """
    try:
        # 🔍 Prepare filter based on status_filter parameter
        filter_params = {}
        if status_filter == "not_done":
            filter_params = {
                "property": "Status",
                "status": {
                    "does_not_equal": "Done"
                }
            }
        elif status_filter == "done":
            filter_params = {
                "property": "Status",
                "status": {
                    "equals": "Done"
                }
            }
        
        # 📤 Query the database
        query_params = {"database_id": ACTIVITY_DB_ID}
        if filter_params:
            query_params["filter"] = filter_params
            
        response = client.databases.query(**query_params)
        
        # 📋 Process results
        pages = []
        for page in response["results"]:
            properties = page["properties"]
            page_dict = {
                "name": properties.get("Name", {}).get('title', [{}])[0].get('plain_text', ""),
                "priority": (properties.get("Priority", {}).get('select') or {}).get('name', ""),
                "deadline": (properties.get("Deadline", {}).get('date') or {}).get('start', ""),
                "status": (properties.get("Status", {}).get('status') or {}).get('name', ""),
                "size": (properties.get("Size", {}).get('select') or {}).get('name', ""),
                "area": (properties.get("Area", {}).get('select') or {}).get('name', ""),
            }
            pages.append(page_dict)
            
        return pages
        
    except Exception as e:
        print(f"❌ Error fetching activities from Notion: {str(e)}")
        return []

# 🎯 Example usage:
# Get all non-completed activities
active_activities = await get_activities()

# Get all completed activities
completed_activities = await get_activities(status_filter="done")

# Get all activities
all_activities = await get_activities(status_filter="all")

### Writing to a Database ✍️

You can create new pages in your database with:
- 📝 Text content
- ✅ Checkboxes
- 📅 Dates
- 👥 People mentions
- And many more property types!

In [7]:
async def write_activity(name, priority=None, deadline=None, status="Not Started", size=None, area=None):
    """
    Create a new activity in the Notion database
    
    Parameters:
        name (str): Name of the activity
        priority (str): Priority level (e.g., "High", "Medium", "Low")
        deadline (str): Deadline date in ISO format (YYYY-MM-DD)
        status (str): Status of the activity (e.g., "Not Started", "In Progress", "Done")
        size (str): Size/effort estimation (e.g., "Small", "Medium", "Large")
        area (str): Area/category of the activity
        
    Returns:
        dict: Created page object or None if failed
    """
    try:
        # 📋 Prepare the properties for the new activity
        properties = {
            "Name": {
                "title": [
                    {
                        "text": {
                            "content": name
                        }
                    }
                ]
            },
            "Status": {
                "status": {
                    "name": status
                }
            }
        }
        
        # Add optional properties if provided
        if priority:
            properties["Priority"] = {
                "select": {
                    "name": priority
                }
            }
            
        if deadline:
            properties["Deadline"] = {
                "date": {
                    "start": deadline
                }
            }
            
        if size:
            properties["Size"] = {
                "select": {
                    "name": size
                }
            }
            
        if area:
            properties["Area"] = {
                "select": {
                    "name": area
                }
            }
            
        # 📤 Create the activity in Notion
        new_page = client.pages.create(
            parent={"database_id": ACTIVITY_DB_ID},
            
            properties=properties
        )
        
        print(f"✅ Successfully created activity: {name}")
        return new_page
        
    except Exception as e:
        print(f"❌ Error creating activity in Notion: {str(e)}")
        return None



In [8]:
# 🎯 Example usage:
new_activity = await write_activity(
    name="Complete Project Documentation",
    priority="High",
    deadline="2024-03-25",
    status="Done",
    size="Medium",
    area="Documentation"
)

✅ Successfully created activity: Complete Project Documentation


## 🎨 Part 5: Understanding Notion Properties

Notion uses different property types for different kinds of data. Here are the main ones you'll work with:

### Common Property Types 📋

1. **Text** 📝
   - Title
   - Rich Text
   - URL

2. **Numbers & Dates** 🔢
   - Number
   - Date
   - Created time
   - Last edited time

3. **Organization** 🗂️
   - Select
   - Multi-select
   - Status
   - Files & Media

4. **People** 👥
   - Person
   - Created by
   - Last edited by

### Property Format Tips 💡

- 📅 Dates should be in ISO format: `2024-03-21`
- ✨ Select options must exist in the database
- 👤 People are referenced by their Notion user IDs
- 🔗 URLs must include `https://` or `http://`

## 🌟 Best Practices & Tips

Here are some pro tips to make your Notion integration awesome:

### Performance Tips 🚀

1. **Batch Operations** 📦
   - Group multiple updates together
   - Use bulk operations when possible
   - Cache results when appropriate

2. **Rate Limits** ⏱️
   - Notion has rate limits
   - Add delay between requests
   - Handle rate limit errors gracefully

### Error Handling 🛡️

Always prepare for these common scenarios:
- 🔒 Authentication errors
- 📡 Network issues
- 🚫 Permission problems
- ⏳ Rate limiting

### Security Best Practices 🔐

1. **Keep Secrets Safe** 
   - Use environment variables
   - Never expose your integration token
   - Regularly rotate tokens

2. **Access Control**
   - Only request necessary permissions
   - Regularly audit database access
   - Remove unused integrations

Remember: A well-structured integration is a happy integration! 🎉